In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# From Sijo VM: Initial exploration: naive XGB 0.32583 on LB
# https://www.kaggle.com/sijovm/sberbank-russian-housing-market/initial-exploration-naive-xgb-0-32583-on-lb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime
#now = datetime.datetime.now()

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
macro = pd.read_csv('../input/macro.csv')

train.sample(3)

In [ ]:
train.columns

In [ ]:
train.shape

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(train.price_doc.values, bins=100, kde=True)
plt.xlabel('Price', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(np.log(train.price_doc.values), bins=100, kde=True)
plt.xlabel('Price', fontsize=12)
plt.show()

In [ ]:
def missing_plot(dataframe, figure_x, figure_y):
    df = dataframe.isnull().sum().reset_index()
    df.columns = ['column_name', 'na_count']
    df = df[df.na_count > 0]
    df = df.sort_values(by=['na_count'], ascending = [False])
    plt.figure(figsize=(figure_x, figure_y))
    sns.barplot(x="na_count", y ="column_name",data = df, orient="h")
    plt.xlabel('Missing count', fontsize=12)
    plt.show()
missing_plot(train, 10,45)


In [ ]:
def corr_plot(dataframe, top_n, target, fig_x, fig_y):
    corrmat = dataframe.corr()
    #top_n - top n correlations +1 since price is included
    top_n = top_n + 1 
    cols = corrmat.nlargest(top_n, target)[target].index
    cm = np.corrcoef(train[cols].values.T)
    f, ax = plt.subplots(figsize=(fig_x,fig_y))
    sns.set(font_scale=1.25)
    cmap = plt.cm.viridis
    hm = sns.heatmap(cm, cbar=False, annot=True, square=True,cmap = cmap, fmt='.2f', annot_kws={'size': 10}, 
                 yticklabels=cols.values, xticklabels=cols.values)
    plt.show()
    return cols
corr_20 = corr_plot(train, 20, 'price_doc', 10,10)

In [ ]:
corr_20.shape

In [ ]:
features_imp = list(corr_20[1:22])
data_train = train[features_imp]
data_test = test[features_imp]
data_train.head()